In [2]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as random
from scipy.signal import convolve
import scipy
from joblib import Parallel, delayed
import time
from scipy import sparse
import json
from tqdm import tqdm, tqdm_notebook

%run ../Scripts/coverage.py
%run ../Scripts/fitness.py
%run ../Scripts/immunity.py
%run ../Scripts/initMethods.py
%run ../Scripts/supMethods.py
%run ../Scripts/mutation.py

In [42]:
params = { #parameters relevant for the equations
    "Nh":        10000000,
    "N0":          100000,
    "R0":              20,
    "M":                1, #Also L, total number of spacers
    "mu":             0.2, #mutation rate
    "gamma_shape":     20, 
    "Np":               0, #Number of Cas Protein
    "dc":               3, #Required number of complexes to activate defence
    "h":                4, #coordination coeff
    "r":             1000, #cross-reactivity kernel
}
sim_params = { #parameters relevant for the simulation (including Inital Valuess)
    "xdomain":                    100,
    "dx":                           1,
    "t0":                           0, 
    "tf":                        1000,
    "dt":                           1,
    "initial_mean_n":           [0,0],
    "initial_var_n":               10,
    "initial_mean_nh":          [0,0],
    "initial_var_nh":              10,
    "n_step_prior":                 5,
    "conv_size":                  100,
    "num_threads":                  4,
    "tail_axis":               [1, 1],
}

In [43]:
def fill_parameters(params, sim_params):
    R0 = params["R0"]
    M = params["M"]
    N0 = params["N0"]
    r = params["r"]

    params["D"] = D = calc_diff_const(params, sim_params)
    inv_v_tau = (np.power(R0, 1/M)-1)/r
    params["s"] = s = M*inv_v_tau

    common_log = 24*np.log(N0*np.power(D*np.power(s,2), 1/3))
    sigma = np.power(D/s, 1/3)*np.power(common_log, 1/6)
    v0 = np.power(s, 1/3)*np.power(D, 2/3)*np.power(common_log, 1/3)
    uc = s*np.power(sigma, 4)/(4*D)

    sim_params["initial_var_n"] = sim_params["initial_var_nh"] = sigma
    params["v0"] = v0
    sim_params["initial_mean_n"] = [0, uc]
    
    return params, sim_params

In [44]:
params, sim_params = fill_parameters(params, sim_params)

In [45]:
n = init_guassian(params["N0"], sim_params, "n")
nh = init_exptail(params["Nh"], params, sim_params, "nh")
kernel_quarter = init_quarter_kernel(params, sim_params)
plt.contour(n.todense(), label = "n", cmap = "Reds")
plt.contour(nh.todense(), label = "nh", cmap = "Blues")
plt.legend()
plt.show()

In [31]:
R0 = params["R0"]
mu = params["mu"]
k = params["gamma_shape"]
M = params["M"]
Nh = params["Nh"]
N0 = params["N0"]
r = params["r"]

tau = M*Nh/N0
params["D"] = D = calc_diff_const(params, sim_params)

In [32]:
inv_v_tau = (np.power(R0, 1/M)-1)/r
params["s"] = s = M*inv_v_tau

In [33]:
common_log = 24*np.log(N0*np.power(D*np.power(s,2), 1/3))
sigma = np.power(D/s, 1/3)*np.power(common_log, 1/6)
v0 = np.power(s, 1/3)*np.power(D, 2/3)*np.power(common_log, 1/3)
uc = s*np.power(sigma, 4)/(4*D)

print(f"sigma: {sigma}")
print(f"v0: {v0}")
print(f"uc: {uc}")

sigma: 5.388598677483459
v0: 0.5517029184325534
uc: 19.07118485006243


In [38]:
sim_params["initial_var_n"] = sim_params["initial_var_nh"] = sigma
params["v0"] = v0
sim_params["initial_mean_n"] = [0, uc]

In [37]:
A = np.power((D*r)/(M*(np.power(R0, 1/M)-1)), 1/3)
B = M*(np.power(R0, 1/M)-1)/r
B = N0*np.power(D, 1/3)*np.power(B, 2/3)
B = np.power(24*np.log(B), 1/6)
print(f"sigma check: {A*B}")

A = np.power(D, 2/3)
A = A*np.power((M*(np.power(R0, 1/M)-1))/r,1/3)
B = M*(np.power(R0, 1/M)-1)/r
B = N0*np.power(D, 1/3)*np.power(B, 2/3)
B = np.power(24*np.log(B), 1/3)
print(f"v0 check: {A*B}")

A = np.power(D, 2/3)
A = (1/4)*np.power((D*r)/(M*(np.power(R0, 1/M)-1)),1/3)
B = M*(np.power(R0, 1/M)-1)/r
B = N0*np.power(D, 1/3)*np.power(B, 2/3)
B = np.power(24*np.log(B), 2/3)
print(f"uc check: {A*B}")

sigma check: 5.388598677483459
v0 check: 0.5517029184325534
uc check: 19.071184850062433
